In [ ]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
from tqdm import tqdm

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Pair Programming Random Forest


### Ana G y Ana C

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Random Forest en este tipo de problemas. 

Los objetivos de este pair programming :

- Ajustad un modelo de Random Forest a nuestros datos.
- Calculad las métricas a nuestro nuevo modelo.
- Comparad las métricas con los modelos hechos hasta ahora. ¿Cuál es mejor?

In [ ]:
df_esta = pd.read_pickle("../data-log/02-df_estandarizadas_codificadas.pickle")
df_esta.head()

In [ ]:
df_noesta = pd.read_pickle("../data-log/02-df_codifcadas_no_estandarizadas.pickle")
df_noesta.head()

In [ ]:
df_bal = pd.read_pickle("../data-log/02-bal_est_cod.pickle")
df_bal.head()

---

# 1. Datos  no estandarizadas, no balanceado

In [ ]:
# separamos los datos en X e y

X1 = df_noesta.drop("TenYearCHD", axis = 1)
y1 = df_noesta["TenYearCHD"]

---

# 2. Estandarizadas y no balanceado

In [ ]:
# separamos los datos en X e y

X2 = df_esta.drop("TenYearCHD", axis = 1)
y2 = df_esta["TenYearCHD"]

---

# 3. Balanceada, estandarizadas

# separamos los datos en X e y

X3 = df_bal.drop("TenYearCHD", axis = 1)
y3 = df_bal["TenYearCHD"]